In [1]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import spacy
# import wordcloud
# import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')
# from collections import Counter

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import os
abspath = os.path.abspath('D:/Box Sync/editorials/scripts/')
os.chdir(abspath)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 150)

print("done")

done


In [2]:
import import_func as imp
import desc_vis as vis
import tags_work as tgs
# import D:/Box Sync/editorials/scripts/import_func as imp
# import D:/Box Sync/editorials/scripts/desc_vis as vis

# Data
elist = "D:/Box Sync/editorials/data/editorials-1.jl"
tags = "D:/Box Sync/editorials/data/by_article_110219.jl"

print("done")

done


Import and process the data from .jl files using import_func functions

In [3]:
elist = "../data/editorials-1.jl"
tags = "../data/by_article_110219.jl"

editorial = imp.import_jl(elist)
tags = imp.import_jl(tags)

out_edi = imp.process(editorial, "editorial")
out_tag = imp.process(tags, "tags")

edi_df = out_edi[0]
edi_dict = out_edi[1]

tag_df = out_tag[0]
tag_dict = out_tag[1]

edi_df.tail()

,headline,preview,authors,date
4,three keys to launching your own lab,"in this letter to young scientists, our columnists share their advice for new faculty members","leah h. somerville, william a. cunningham, et al.",2019-08-14
3,ultrarunning made me a more resilient scientist,"lessons learned from a running group helped this grad student redefine success, both professiona...",mikel haggadone,2019-08-15
2,why some professors welcome new lab members with clear expectations—in writing,documents like these can help labs run more smoothly,katie langin,2019-08-19
1,how do you doogie? science’s infatuation with young prodigies,our experimental error columnist reflects on the allure of youth,adam ruben,2019-08-21
0,how i conquered my fear of public speaking and learned to give effective presentations,this postdoc learned four key lessons that helped her engage with her audiences,mathilde his,2019-08-22


Clean the dataframes
* make a sequential date column for both dates and months
* indicate whether posts are columns or not (because early posts do not have a column tag, posts by authors that appear more than [5] times will be considered columns

In [4]:
edi_df = imp.seq_dates(edi_df, "editorial")
tag_df = imp.seq_dates(tag_df, "tags")

tags_df = imp.id_columns(tag_df)

tags_df = tgs.seperate_tags(tags_df)

len(tags_df)


23104

## Tags work ##

In [8]:

inc_dict = tgs.tag_incidence(tags_df, lifespan = True, id_col_tag = True, binary = True)

#print(inc_dict["working life"]["overall"])

col_tag_todf = {}

for i in tags_df.index.values:
    w_tag = tags_df.loc[i, "tag"]
    col_tag_todf[w_tag] = inc_dict[w_tag]["overall"]["col_tag"]

tags_df["col_tag"] = tags_df["tag"].map(col_tag_todf) 

tags_df.tail()

,headline,authors,date,time,date_seq,month_seq,year,n_posts_author,column1,column2,variable,tag,col_tag
189977,getting credit for peer review,sandeep ravindran,2016-02-08,3:15 pm,7070,242,2016,1,no,no,30,india,no
196112,want a letter? you write it for me,roger s. day,2016-01-07,2:30 pm,7038,241,2016,1,yes,yes,31,oncology,no
196113,getting credit for peer review,sandeep ravindran,2016-02-08,3:15 pm,7070,242,2016,1,no,no,31,united states,no
202248,want a letter? you write it for me,roger s. day,2016-01-07,2:30 pm,7038,241,2016,1,yes,yes,32,other fields,no
208384,want a letter? you write it for me,roger s. day,2016-01-07,2:30 pm,7038,241,2016,1,yes,yes,33,americas,no


In [14]:
# tags incidence dataframe
tag_inc_df = {"year":[],"tag":[],"incidence":[]}
tag_inc_counting = {}
# tag_inc_counting = {tag: {year:int, year: int, year: int}}

# count incidences of each tag per year
for key in [key for key in inc_dict.keys()]:
    # print(key)
    if inc_dict[key]["overall"]["col_tag"] == "no": # filter out tags that are probs identifying columns
        if key in tag_inc_counting.keys():
            for post in inc_dict[key]["overall"]["span"]:
                w_year = imp.cumul_to(post[1], "m")
                if w_year in tag_inc_counting[key].keys():
                    tag_inc_counting[key][w_year] += 1
        else:
            tag_inc_counting[key] = {}
            for post in inc_dict[key]["overall"]["span"]:
                tag_inc_counting[key][imp.cumul_to(post[1], "m")] = 1

# convert counting dictionary to dictionary that can be made into a df
for key in tag_inc_counting.keys():
    for year in tag_inc_counting[key].keys():
        tag_inc_df["year"].append(year)
        tag_inc_df["tag"].append(key)
        tag_inc_df["incidence"].append(tag_inc_counting[key][year])

tag_inc_df = pd.DataFrame(tag_inc_df)
tag_inc_df.tail()

,year,tag,incidence
996,2016,structural biology,1
997,2016,toxicology,1
998,2016,veterinary medicine,1
999,2016,virology,1
1000,2016,biophysics,1


Descriptive for tags
* how many tags total
* average number of tags per post


In [0]:
# total tags
len(inc_dict)

# average tags per post
np.average(tags_df.groupby(by = "headline").size())


# Publication Rates

In [0]:
# Density of all posts by Sequential date
sns.distplot(editorial["date_seq"])

In [0]:
# Posts each month
